## Vivado / Vitis HLS Backend

In [ ]:
# import allo
import allo
from allo.ir.types import float32
import numpy as np

In [ ]:
# parameters for matrix dimensions
M, N, K = 32, 32, 32

# define gemm function
def gemm(A: float32[M, K], B: float32[K, N]) -> float32[M, N]:
  C: float32[M, N]
  for i, j in allo.grid(M, N):
    # separately annotate k for optimizations 
    for k in allo.reduction(K):
      C[i, j] += A[i, k] * B[k, j]
  return C

In [ ]:
# create schedule
s = allo.customize(gemm)

# reorder the inner and middle loops
s.reorder("k", "j")
print(s.module)

In [ ]:
# create a row buffer for the output tensor C
s.buffer_at(s.C, axis="i")
print(s.module)

In [ ]:
# pipeline j loop
s.pipeline("j")
print(s.module)

In [ ]:
# generate HLS code
code = s.build(target="vhls")
print(code)

In [ ]:
# compile program into RTL for cosim
mod = s.build(target="vitis_hls", mode="hw_emu", project="gemm.prj")

In [ ]:
# simulate the RTL
np_A = np.random.random((M, K)).astype(np.float32)
np_B = np.random.random((K, N)).astype(np.float32)
allo_C = np.zeros((M, N), dtype=np.float32)

mod(np_A, np_B, allo_C)

np.testing.assert_allclose(allo_C, np.matmul(np_A, np_B), rtol=1e-5, atol=1e-5)